#importing libraries

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime

In [15]:
import matplotlib.pyplot as plt
plt.style.use('default')

In [16]:
import seaborn as sns
import re

In [35]:
from nltk import bigrams,trigrams,word_tokenize,FreqDist
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk.sentiment.vader

In [18]:
from wordcloud import wordcloud
from gensim.utils import simple_preprocess
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy

In [19]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection,svm
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD

In [20]:
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\chinm\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chinm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chinm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chinm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\chinm\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Read the data

In [21]:
tweet_data=pd.read_csv('tweets_v8.csv')
tweet_data
# tweet_data.shape

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,source,is_retweet
0,the _ûndër-ratèd niggáh👊🏾,NaN,@ManUtd die hard❤️❤️💪🏿💪🏿\n\n\nYOLO\n\n\nJ'ai b...,2019-09-06 19:24:57+00:00,581,1035,8922,False,2021-10-06 12:05:38+00:00,When life hits and the same time poverty strik...,Twitter for Android,False
1,Best uncle on planet earth,NaN,NaN,2013-05-08 19:35:26+00:00,741,730,8432,False,2021-10-06 12:05:22+00:00,That marble episode of #SquidGame ruined me. 😭😭😭,Twitter for Android,False
2,marcie,NaN,animal crossing. chicken nuggets. baby yoda. s...,2009-02-21 10:31:30+00:00,562,1197,62732,False,2021-10-06 12:05:22+00:00,#Squidgame time,Twitter Web App,False
3,YoMo.Mdp,Any pronouns,Where the heck is the karma\nI'm going on my s...,2021-02-14 13:21:22+00:00,3,277,1341,False,2021-10-06 12:05:04+00:00,//Blood on 1st slide\nI'm joining the squidgam...,Twitter Web App,False
4,Laura Reactions,France,I talk and I make reactions videos about shows...,2018-12-19 20:38:28+00:00,330,152,2278,False,2021-10-06 12:05:00+00:00,"The two first games, players were killed by th...",Twitter Web App,False
...,...,...,...,...,...,...,...,...,...,...,...,...
80014,Sale X Low,USA,"Sale X Low, the cheapest low price online stor...",2021-04-03 20:49:35+00:00,12,49,7,False,2021-10-28 13:46:26+00:00,Yes Yes Yes\n\n@ArianaGrande\n#Squidgame\n htt...,SocialRabbit Plugin,False
80015,RevAAA,NaN,Review Anything Anyone Anywhere,2010-11-24 21:48:53+00:00,6907,0,0,False,2021-10-28 13:45:38+00:00,SQUID GAME was reviewed on RevAAA #squidgame \...,Twitter for iPhone,False
80016,Omo K-Pop News,NaN,OMO K-Pop\nNews and Updates,2021-09-22 12:01:24+00:00,152,807,593,False,2021-10-28 13:45:00+00:00,Back &amp; Forth Between ‘Squid Game’ Creator ...,Twitter Web App,False
80017,🎃Pumpkin💀Queen 🕸,Halloweentown 🎃,Creator of Stuff | Horror fan | Traveler | Bad...,2009-02-19 17:09:28+00:00,8048,8822,68980,False,2021-10-28 13:44:57+00:00,So what sort of games do you think they'll pla...,Twitter for Android,False


Sentiment Analysis

Lexicon Sentiment Analysis -TextBlob

In [22]:
stop_words=stopwords.words('english')
stop_words.extend(['squid','game'])

def clean_tweet(text,full_clean=False):
    le =WordNetLemmatizer()
    if type(text)==np.float64:
        return ""
    text=re.sub("'","",text)
    text = re.sub("\n", "", text)
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub('[()!?]', ' ', text)
    if full_clean:
      text = text.lower()
      text = re.sub("[^a-z0-9]"," ", text)
      text = [le.lemmatize(w) for w in text.split() if not (w in stop_words) and (len(w)>2)]
    else:
        text = re.sub("[^A-Za-z0-9]"," ", text)
        text = [le.lemmatize(w) for w in text.split() if (len(w)>2)]
    text = " ".join(word for word in text)
    return text

In [23]:
def textblob_sentiment(text):
    analysis=TextBlob(clean_tweet(text))
    if analysis.sentiment.polarity > 0:
        return "Positive"
    elif analysis.sentiment.polarity < 0:
        return "Negative"
    else:
        return "Neutral"
    

In [24]:
def vader_sentiment(text):
    obj=SentimentIntensityAnalyzer()
    sentiment_dict=obj.polarity_scores(clean_tweet(text))
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
    elif sentiment_dict['compound'] <=-0.05:
        return "Negative"
    else:
        return "Neutral"

In [25]:
def sentiment_scores(text):
    obj=SentimentIntensityAnalyzer()
    sentiment_dict=obj.polarity_scores(clean_tweet(text))
    sentiment_dict['neg']=sentiment_dict['neg']*100
    sentiment_dict['neu']= sentiment_dict['neu']*100
    sentiment_dict['pos']=sentiment_dict['pos']*100
    lst=[sentiment_dict['neg'],sentiment_dict['neu'],sentiment_dict['pos']]
    return max(lst)


In [26]:
# def conf_matrix(x):
#   if x[1] == 1 and x[2] == 1:
#     return 'True Positive'
#   elif x[1] == 1 and x[2] == -1:
#     return 'False Ngative'
#   elif x[1] == -1 and x[2] == 1:
#     return 'False Positive'
#   elif x[1] == -1 and x[2] == -1:
#     return 'True Negative'
#   else:
#     return 0
    
# tweet_data['Conf_Matrix'] = tweet_data['text'].apply(lambda x: conf_matrix(clean_tweet(x)), axis=1)


In [27]:
I=75234
tweet_data['text'][I]

'Just finished watching this ⭕\n"Circle" from Squid Game\n\n#artbyinyu #chibi #chibiart #SquidGame #squidgamefanart… https://t.co/06lbaZBiiH'

In [28]:
cleaned_tweet=clean_tweet(tweet_data['text'][I])
cleaned_tweet

'Just finished watching this Circle from Squid Game'

In [29]:
textblob_sentiment(cleaned_tweet)

'Negative'

In [30]:
sentiment_scores(cleaned_tweet)

100.0

In [31]:
tweet_data['TextBlob_Sentiment']=tweet_data['text'].apply(lambda x:textblob_sentiment(clean_tweet(x)))

In [39]:
tweet_data['VADER_Sentiment']=tweet_data['text'].apply(lambda x:vader_sentiment(clean_tweet(x)))

In [37]:
tweet_data['Sentiment_score']=tweet_data['text'].apply(lambda x:sentiment_scores(clean_tweet(x)))

In [ ]:
tweet_data['TextBlob_Sentiment']

0        Negative
1         Neutral
2         Neutral
3        Negative
4        Negative
           ...   
80014     Neutral
80015    Negative
80016    Negative
80017    Negative
80018     Neutral
Name: TextBlob_Sentiment, Length: 80019, dtype: object

In [40]:
tweet_data

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,source,is_retweet,TextBlob_Sentiment,Sentiment_score,VADER_Sentiment
0,the _ûndër-ratèd niggáh👊🏾,NaN,@ManUtd die hard❤️❤️💪🏿💪🏿\n\n\nYOLO\n\n\nJ'ai b...,2019-09-06 19:24:57+00:00,581,1035,8922,False,2021-10-06 12:05:38+00:00,When life hits and the same time poverty strik...,Twitter for Android,False,Negative,60.4,Negative
1,Best uncle on planet earth,NaN,NaN,2013-05-08 19:35:26+00:00,741,730,8432,False,2021-10-06 12:05:22+00:00,That marble episode of #SquidGame ruined me. 😭😭😭,Twitter for Android,False,Neutral,50.8,Negative
2,marcie,NaN,animal crossing. chicken nuggets. baby yoda. s...,2009-02-21 10:31:30+00:00,562,1197,62732,False,2021-10-06 12:05:22+00:00,#Squidgame time,Twitter Web App,False,Neutral,100.0,Neutral
3,YoMo.Mdp,Any pronouns,Where the heck is the karma\nI'm going on my s...,2021-02-14 13:21:22+00:00,3,277,1341,False,2021-10-06 12:05:04+00:00,//Blood on 1st slide\nI'm joining the squidgam...,Twitter Web App,False,Negative,71.9,Negative
4,Laura Reactions,France,I talk and I make reactions videos about shows...,2018-12-19 20:38:28+00:00,330,152,2278,False,2021-10-06 12:05:00+00:00,"The two first games, players were killed by th...",Twitter Web App,False,Negative,58.8,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80014,Sale X Low,USA,"Sale X Low, the cheapest low price online stor...",2021-04-03 20:49:35+00:00,12,49,7,False,2021-10-28 13:46:26+00:00,Yes Yes Yes\n\n@ArianaGrande\n#Squidgame\n htt...,SocialRabbit Plugin,False,Neutral,100.0,Positive
80015,RevAAA,NaN,Review Anything Anyone Anywhere,2010-11-24 21:48:53+00:00,6907,0,0,False,2021-10-28 13:45:38+00:00,SQUID GAME was reviewed on RevAAA #squidgame \...,Twitter for iPhone,False,Negative,100.0,Neutral
80016,Omo K-Pop News,NaN,OMO K-Pop\nNews and Updates,2021-09-22 12:01:24+00:00,152,807,593,False,2021-10-28 13:45:00+00:00,Back &amp; Forth Between ‘Squid Game’ Creator ...,Twitter Web App,False,Negative,100.0,Neutral
80017,🎃Pumpkin💀Queen 🕸,Halloweentown 🎃,Creator of Stuff | Horror fan | Traveler | Bad...,2009-02-19 17:09:28+00:00,8048,8822,68980,False,2021-10-28 13:44:57+00:00,So what sort of games do you think they'll pla...,Twitter for Android,False,Negative,80.6,Positive


In [ ]:
data = tweet_data[['user_location','date','text','source','user_verified','is_retweet','TextBlob_Sentiment']]
data

,user_location,date,text,source,user_verified,is_retweet,TextBlob_Sentiment
0,NaN,2021-10-06 12:05:38+00:00,When life hits and the same time poverty strik...,Twitter for Android,False,False,Negative
1,NaN,2021-10-06 12:05:22+00:00,That marble episode of #SquidGame ruined me. 😭😭😭,Twitter for Android,False,False,Neutral
2,NaN,2021-10-06 12:05:22+00:00,#Squidgame time,Twitter Web App,False,False,Neutral
3,Any pronouns,2021-10-06 12:05:04+00:00,//Blood on 1st slide\nI'm joining the squidgam...,Twitter Web App,False,False,Negative
4,France,2021-10-06 12:05:00+00:00,"The two first games, players were killed by th...",Twitter Web App,False,False,Negative
...,...,...,...,...,...,...,...
80014,USA,2021-10-28 13:46:26+00:00,Yes Yes Yes\n\n@ArianaGrande\n#Squidgame\n htt...,SocialRabbit Plugin,False,False,Neutral
80015,NaN,2021-10-28 13:45:38+00:00,SQUID GAME was reviewed on RevAAA #squidgame \...,Twitter for iPhone,False,False,Negative
80016,NaN,2021-10-28 13:45:00+00:00,Back &amp; Forth Between ‘Squid Game’ Creator ...,Twitter Web App,False,False,Negative
80017,Halloweentown 🎃,2021-10-28 13:44:57+00:00,So what sort of games do you think they'll pla...,Twitter for Android,False,False,Negative
